# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import _script_utils
import _data_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
### Read in data (First round)
# proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))
# proj.drop(columns =['unnamed:_0'], axis=1, inplace=True)
# proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')


In [6]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230309_ToDLA.xlsx",))


In [7]:
proj.sample(2)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
678,44728,Y001,NATIONAL HIGHWAY PERF IIJA,0801121,0420000288S,ALAMEDA COUNTY IN BERKELEY AND ALBANY FROM 0.6 MILE NORTH OF UNIVERSITY AVENUE OVERCROSSING TO 0.3 MILE SOUTH OF EL CERRITO SEPARATION AND OVERHEAD E,1,Cong Dist 13,Active,"ON STATE ROUTE: 80. ALAMEDA COUNTY IN BERKELEY AND ALBANY FROM 0.6 MILE NORTH OF UNIVERSITY AVENUE OVERCROSSING TO 0.3 MILE SOUTH OF EL CERRITO SEPARATION AND OVERHEAD EXCAVATION, HMA, STRUCTURAL CONCRETE, MINOR CONCRETE AND DRAINAGE.",17,Construction Engineering,3347000.0,2481500.0,S MTC
695,44728,Y001,NATIONAL HIGHWAY PERF IIJA,5298030,0813000018L,"NORTH 1ST AVENUE OVER THE BNSF RAILROAD FROM CLARK STREET TO CROOKS AVENUE, BR. NO. 54C-0088 BRIDGE REPLACEMENT",71,Cong Dist 8,Active,"NORTH 1ST AVENUE OVER THE BNSF RAILROAD FROM CLARK STREET TO CROOKS AVENUE, BR. NO. 54C-0088 BRIDGE REPLACEMENT",16,Right of Way,2000000.0,1770600.0,L5298SCAG


In [8]:
proj.county_code.value_counts()

37     284
23     124
999    105
19      96
67      84
59      81
71      81
73      77
65      71
1       66
85      61
7       59
83      58
87      56
45      53
81      47
17      46
29      43
79      42
53      41
13      38
97      37
95      36
41      35
89      35
47      34
107     34
77      34
55      32
61      30
115     26
99      25
57      25
9       17
75      14
105     14
39      13
91      12
113     11
15      11
111     11
33      10
69       9
103      9
93       8
43       8
11       7
31       7
63       6
21       6
27       5
109      5
5        3
35       3
101      3
51       2
25       1
Name: county_code, dtype: int64

In [9]:
# proj.congressional_district.

In [10]:
## move to _data_utils
# def add_new_codes(df):
#     new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))
#     code_map = dict(new_codes[['iija_program_code', 'new_description']].values)
    
#     df['program_code_description'] = df.program_code.map(code_map)
#     proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].astype(str)
    
#     return df


In [11]:
proj2 = _data_utils.add_new_codes(proj)

In [12]:

    ## function that adds known agency name to df 
df = _script_utils.identify_agency(proj2, 'summary_recipient_defined_text_field_1_value')

In [13]:
# df.sample(3)

#### Breaking down Idenitfy Agency 

In [14]:
# #add project information for all projects
# def identify_agency1(df, identifier_col):
#     #projects wtih locodes
#     locode_proj = ((df[~df[identifier_col].str.contains(" ")]))
#     locode_proj = locode_proj>>filter(_[identifier_col]!='None')
    
#     locode_proj = (_data_utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))
    

#     #projects with no locodes
#     no_locode = ((df[df[identifier_col].str.contains(" ")]))
#     no_entry = df>>filter(_[identifier_col]=='None')
    
#     #concat no locodes and those with no entry
#     no_locode = pd.concat([no_locode, no_entry])
    
#     #add county codes:
#     county_base = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of County.xlsx", sheet_name='County', header=[1]))
#     county_base.drop(columns =['unnamed:_0', 'unnamed:_4'], axis=1, inplace=True)
#     county_base['county_description'] = county_base['county_description'] + " County"
    
#     #read in locode info
#     locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))
#     county_district = locodes>>group_by(_.district, _.county_name)>>count(_.county_name)>>select(_.district, _.county_name)>>filter(_.county_name!='Multi-County', _.district !=53)
    
#     # merge county information to add districts
#     county_info = (pd.merge(county_base, county_district, how='left', left_on= 'county_description', right_on = 'county_name'))
#     county_info.drop(columns =['county_name'], axis=1, inplace=True)
    
#     mapping1 = dict(county_info[['county_code', 'county_description']].values)
#     mapping2 = dict(county_info[['county_code', 'recipient_name']].values)
#     mapping3 = dict(county_info[['county_code', 'district']].values)
    
#     no_locode['county_description'] = no_locode.county_code.map(mapping1)
#     no_locode['recipient_name'] = no_locode.county_code.map(mapping2)
#     no_locode['district'] = no_locode.county_code.map(mapping3)
    
#     no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

#     full_df = pd.concat([locode_proj, no_locode])
    
#     full_df.loc[full_df.county_name == "Statewide County", 'county_name'] = "Statewide"
    
#     full_df['implementing_agency'] = full_df['implementing_agency'].fillna(value='Unknown')
#     full_df['county_name'] = full_df['county_name'].fillna(value='Unknown')

    
#     return full_df


In [15]:
# #add project information for all projects

# locode_proj = ((proj2[~proj2['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# locode_proj = locode_proj>>filter(_['summary_recipient_defined_text_field_1_value']!='None')
    
# locode_proj = (_data_utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))


In [16]:

# #projects with no locodes
# no_locode = ((proj2[proj2['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# no_entry = proj2>>filter(_['summary_recipient_defined_text_field_1_value']=='None')
    
# #concat no locodes and those with no entry
# no_locode = pd.concat([no_locode, no_entry])
    

In [17]:
# len(no_locode)

In [18]:
# no_locode.sample()

In [19]:
# locode_proj.sample()

In [20]:
# #add county codes:
# county_base = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of County.xlsx", sheet_name='County', header=[1]))
# county_base.drop(columns =['unnamed:_0', 'unnamed:_4'], axis=1, inplace=True)
# county_base['county_description'] = county_base['county_description'] + " County"

# #read in locode info
# locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))
# county_district = locodes>>group_by(_.district, _.county_name)>>count(_.county_name)>>select(_.district, _.county_name)>>filter(_.county_name!='Multi-County', _.district !=53)

In [21]:
# # merge county information to add districts
# county_info = (pd.merge(county_base, county_district, how='left', left_on= 'county_description', right_on = 'county_name'))
# county_info.drop(columns =['county_name'], axis=1, inplace=True)
    
# mapping1 = dict(county_info[['county_code', 'county_description']].values)
# mapping2 = dict(county_info[['county_code', 'recipient_name']].values)
# mapping3 = dict(county_info[['county_code', 'district']].values)

In [22]:
   
# no_locode['county_description'] = no_locode.county_code.map(mapping1)
# no_locode['recipient_name'] = no_locode.county_code.map(mapping2)
# no_locode['district'] = no_locode.county_code.map(mapping3)
    

In [23]:
# no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

# full_df = pd.concat([locode_proj, no_locode])

In [24]:
# no_locode.sample()

In [25]:
# full_df.loc[full_df.county_name == "Statewide County", 'county_name'] = "Statewide"


In [26]:
# (full_df
#  >>filter(_.summary_recipient_defined_text_field_1_value.str.contains('S '))).summary_recipient_defined_text_field_1_value.value_counts()

In [27]:
# (full_df
#  >>filter(_.summary_recipient_defined_text_field_1_value.str.contains('S '))
#  >>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.county_code, _.county_name, _.project_description, _.project_title, _.program_code_description)
#  >>filter(_.summary_recipient_defined_text_field_1_value != ("S    ER NONE"))
# )


In [28]:
### hash out to test the find_alternative_name function on full_df.
# state_test = (full_df>>filter(_.county_name==('Statewide')) >>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.county_code, _.county_name, _.project_description, _.project_title, _.program_code_description)
# )

#### Breakdown agency identify

In [29]:
# def read_in_locodes():
#     locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))
#     locodes['agency_name'] = locodes['agency_name'].str.upper()
#     locode_names = sorted(list(locodes['agency_name'].unique()), reverse=True)    
    
#     ## append list so that "Crosswalk" does not select "Ross"
#     locode_names.remove('ROSS')
#     locode_names.append("STATE PARKS")
#     locode_names.append("SACOG - Sacramento Area Council of Governments")
#     locode_names.append("SACOG")
#     locode_names.append("MTC")
#     locode_names.append("MTC - Metropolitan Transportation Commission")
#     locode_names.append("Statewide")
    
#     return locode_names

In [30]:
# def find_alternative_name(df, desc_col, new_col_name):
    
#     #read in locode names to get list to find in description column
#     locode_names = read_in_locodes()
    
#     ### https://stackoverflow.com/questions/68869434/create-an-pandas-column-if-a-string-from-a-list-matches-from-another-column
#     df[new_col_name] = df[desc_col].map(lambda s: next((name for name in locode_names if name in s), ""))
#     df[new_col_name] = df[new_col_name].str.title()
    
#     return df

In [31]:
## run the read_in_locodes, and find_alterntaive_names functions from the section Next Statewide

# test_title_df = find_alternative_name(full_df, 'project_description', "name_in_desc")

In [32]:
# test_title_df>>filter(_.name_in_desc == "")>>filter(_.county_name=="Statewide")

In [33]:
# ## changing the title function to include the new description col
# def add_new_title(df, first_col_method, second_col_type, third_col_name, alt_col_name, alt_in_proj_desc):
#     """
#     Function to add new title. 
#     Expected output example: "New Bike Lane in Eureka"
#     """    

    
#     def return_name(df):
        
#         if (df[third_col_name] == "California") & (df[alt_col_name] == "Statewide") & (df[alt_in_proj_desc] != ""):
#             return (df[first_col_method] + " " + df[second_col_type] +" in " + df[alt_in_proj_desc])
        
#         elif (df[third_col_name] == "California") & (df[alt_col_name] == "Statewide") & (df[alt_in_proj_desc] == ""):
#             return (df[first_col_method] + " " + df[second_col_type] +" " + df[alt_col_name])
                    
#         elif (df[third_col_name]== "Unknown"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_in_proj_desc])
        
#         elif (df[third_col_name] == "California"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_col_name])
        
#         elif (df[third_col_name] == "Metropolitan Transportation Commission"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in the " + df[third_col_name] + " Region")
        
#         elif (df[third_col_name] == "Los Angeles County Metropolitan Transportation Authority"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in the " + df[third_col_name] + " Region")
        
#         elif (df[third_col_name] == "Caltrans") & (df[alt_in_proj_desc] != ""):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_in_proj_desc])
        
#         elif (df[third_col_name] == "Caltrans") & (df[alt_in_proj_desc] == ""):
#             return (df[third_col_name] + " " + df[first_col_method] + " " + df[second_col_type])
        
#         elif (df[third_col_name] != "California"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[third_col_name])
        

#         return df

#     df['project_name_new'] = df.apply(return_name, axis = 1)
    
#     return df

In [34]:
# ## adding in the project title new types (in the broader add title funtion)

# full_df_copy = _script_utils.add_description(full_df, 'project_title')
    
# #remove project method column values so that the title function wont double count
# full_df_copy.loc[full_df_copy['project_type'] == 'Project', 'project_method'] = ""
# full_df_copy['project_type'] = full_df_copy['project_type'].replace('Project', np.NaN)
    
# #update for the projects not in the first round of descriptions
# full_df_copy =  _script_utils.add_description_4_no_match(full_df_copy, 'improvement_type_description')
# ## proj_unique_cat_title = update_no_matched(proj_unique_cat, "project_type", 'improvement_type_description', 'program_code_description')
    
# #fill nan values in 'Project_type' with values from 'project_type2' from add_description_4_no_match function
# full_df_copy['project_type'] = full_df_copy['project_type'].fillna(full_df_copy['project_type2'])

In [35]:
 # full_df_copy[['county_code', 'improvement_type',
 #     'implementing_agency_locode', 'district',
 #     'program_code_description', 'recipient_project_number',
 #          "project_method", "project_type", "implementing_agency", "county_name", "name_in_desc"]] = full_df_copy[['county_code', 'improvement_type',
 #                                                                     'implementing_agency_locode', 'district',
 #                                                                     'program_code_description', 'recipient_project_number',
 #                                                                    "project_method", "project_type", "implementing_agency", "county_name", "name_in_desc"]].astype(str)

In [36]:
# full_df_title = add_new_title(full_df_copy, "project_method", "project_type", "implementing_agency", "county_name", "name_in_desc")

In [37]:
# full_df_copy.sample()

In [38]:
# full_df>>filter(_.name_in_desc == "")>>filter(_.county_name=="Statewide")

In [39]:
# full_df_copy>>filter(_.name_in_desc=="")

In [40]:
# full_df_copy>>filter(_.county_name=="Statewide")>>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.county_name, _.project_title, _.project_description, _.name_in_desc,
#                                                          _.project_method, _.project_type, _.project_type2, _.project_name_new)

In [41]:

    
    
# full_df['implementing_agency'] = full_df['implementing_agency'].fillna(value='Unknown')
# full_df['county_name'] = full_df['county_name'].fillna(value='Unknown')



##### Check unknowns

In [42]:
len(df>>filter(_.implementing_agency=='Unknown'))



1

In [43]:
len(df>>filter(_.county_name=='Unknown'))

1

In [44]:
df>>filter(_.implementing_agency=="Unknown")

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
706,44827,YR10,Recreational Trails Program,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999,Cong Dist 0,Active,"2023 Rec Trails Project: State Parks Incurred Administrative Expenses. State costs incurred administering the RTP such as staff time, meeting and travel costs and attendance at trail-related training sessions and conferences, Trail Advisory committee costs including meeting and travel costs for committee members, trail conference support, etc.",44,Other,274727.3,241760.0,nan,NaN,Unknown,NaN,Unknown,NaN,NaN


In [45]:
# df>>filter(_.project_title.str.contains("Administrative Expense"))

#### Reformat Columns

To change: 
* ~~District: 2 digit / integer~~
* ~~Congressional District number~~
* Add Progam Code Description and amount to a new project description

In [46]:
df = _data_utils.change_col_to_integer(df, "congressional_district")

In [47]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
80,44599,ER01,Emergency Supplement Funding,37Y4002,0320000035S,"IN AND NEAR PARADISE, FROM 0.3 MILE SOUTH OF AIRPORT ROAD TO 0.2 MILE NORTH OF OLD CLARK ROAD. STABILIZE THE FIRE DAMAGED CUT SLOPES, WIDEN SHOULDERS",7,1,Active,"IN AND NEAR PARADISE, FROM 0.3 MILE SOUTH OF AIRPORT ROAD TO 0.2 MILE NORTH OF OLD CLARK ROAD. STABILIZE THE FIRE DAMAGED CUT SLOPES, WIDEN SHOULDERS TO CREATE CATCHMENT AREA FOR ROCKFALL DEBRIS, AND IMPROVE DRAINAGE SYSTEMS.",43,Utilities,5190.0,4594.0,S BCAG,NaN,California,3.0,Butte County,NaN,NaN


In [48]:
## move to _data_utils
# def change_col_to_integer(df, col):
    
#     df[col] = df[col].str.split(' ').str[-1]
    
#     return df

In [49]:

# (df>>select(_.congressional_district, _.congressional_district_2)).sample(20)

In [50]:
proj2>>filter(_.summary_recipient_defined_text_field_1_value.str.contains("S "))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
0,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,1983100.77,419227.50,S ER NONE
1,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,793240.31,184329.22,S ER NONE
2,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,16899.23,3572.50,S ER NONE
3,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,6759.69,1570.78,S ER NONE
4,44579,ER01,Emergency Supplement Funding,39BD001,0719000177S,"IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",37,Cong Dist 27,Active,"ON STATE ROUTE: 39. IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,500000.00,61000.00,S ER NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2178,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,Cong Dist 1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,6,4R - Restoration & Rehabilitation,21004300.00,18412317.60,S NON-MPO
2179,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,Cong Dist 1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,17,Construction Engineering,2686000.00,2377900.00,S NON-MPO
2180,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,Cong Dist 1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTE

In [51]:
df>>filter(_.summary_recipient_defined_text_field_1_value.str.contains("S "))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
0,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,1983100.77,419227.50,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
1,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,793240.31,184329.22,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
2,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,16899.23,3572.50,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
3,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,6759.69,1570.78,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
4,44579,ER01,Emergency Supplement Funding,39BD001,0719000177S,"IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",37,27,Active,"ON STATE ROUTE: 39. IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,500000.00,61000.00,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2178,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,6,4R - Restoration & Rehabilitation,21004300.00,18412317.60,S NON-MPO,NaN,California,2.0,Plumas County,NaN,NaN
2179,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,17,Construction Engineering,2686000.00,2377900.00,S NON-MPO,NaN,California,2.0,Plumas County,NaN,NaN
2180,44993,ER01,Em

In [52]:
## move to utils,
# def add_new_description_col(df):
#     df["obligations_amount_string"] = df["obligations_amount"].astype(str)
    
#     df["new_description_col"] = df["program_code_description"] + " for $" + df["obligations_amount_string"]
    
#     df.drop(columns =['obligations_amount_string'], axis=1, inplace=True)
    
#     return df

In [53]:
# (add_new_description_col(df))>>filter(_.program_code_description.notnull())

In [54]:
df.program_code_description.value_counts()

Emergency Supplement Funding                                          590
National Highway Performance Program (NHPP)                           493
Surface Transportation Block Grant                                    386
Highway Safety Improvement Program (HSIP)                             377
Congestion Mitigation & Air Quality Improvement                       104
Transportation Alternatives                                            48
Projects to Reduce PM 2.5 Emissions                                    42
Section 164 Penalties - Use for HSIP Activities                        37
Bridge Formula Program                                                 33
Metropolitan Planning Program                                          19
Safe and Accessible Transportation Options - Metropolitan Planning     18
NHPP Exempt                                                            12
Railway-Highway Crossings Program                                       7
National Highway Freight Program (NHFP

## Test & Export

In [55]:
# GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [56]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

/home/jovyan/data-analyses/dla/iija/_script_utils.py:408: FutureWarning: The default value of regex will change from True to False in a future version.


In [57]:
agg.project_number.value_counts()

0010203    3
00PE023    3
0010206    3
0010209    3
5002200    3
          ..
0010176    1
32D0002    1
38M0003    1
5007090    1
P138066    1
Name: project_number, Length: 1054, dtype: int64

In [58]:
agg>>filter(_.project_number=="0010203")

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
679,44826,0010203,California,S SACOG,Y410,Safe and Accessible Transportation Options - Metropolitan Planning,0022000348S,18,Planning,OWP 22/23 SACRAMENTO REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR SACRAMENTO AREA COUNCIL OF GOVERNMENTS (SACOG) OVERALL WORK PROGRAM (OWP) F,103157,|0|,|nan|,999,Statewide,nan,None,None,Planning and Research in Sacramento Area Council Of Governments,"Planning and Research in Sacramento Area Council Of Governments, part of the Safe and Accessible Transportation Options - Metropolitan Planning. (Federal Project ID: 0010203)."
680,44826,0010203,California,S SACOG,Y450,Metropolitan Planning Program,0022000348S,18,Planning,OWP 22/23 SACRAMENTO REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR SACRAMENTO AREA COUNCIL OF GOVERNMENTS (SACOG) OVERALL WORK PROGRAM (OWP) F,3644977,|0|,|nan|,999,Statewide,nan,None,None,Planning and Research in Sacramento Area Council Of Governments,"Planning and Research in Sacramento Area Council Of Governments, part of the Metropolitan Planning Program. (Federal Project ID: 0010203)."
681,44826,0010203,California,S SACOG,Y550,State Planning and Research,0022000348S,18,Planning,OWP 22/23 SACRAMENTO REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR SACRAMENTO AREA COUNCIL OF GOVERNMENTS (SACOG) OVERALL WORK PROGRAM (OWP) F,500000,|0|,|nan|,999,Statewide,nan,None,None,Planning and Research in Sacramento Area Council Of Governments,"Planning and Research in Sacramento Area Council Of Governments, part of the State Planning and Research. (Federal Project ID: 0010203)."


In [59]:
(agg>>filter(_.county_name=="Statewide"))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
19,44587,39AZ001,California,S ER NONE,ER01,Emergency Supplement Funding,0119000058S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN DEL NORTE COUNTY AT VARIOUS LOCATIONS REPAIR ROADWAY AND CULVERT,2638100,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Del Norte County,"Road Restoration & Rehabilitation in Del Norte County, part of the Emergency Supplement Funding. (Federal Project ID: 39AZ001)."
163,44676,000C533,California,S MTC,Y001,National Highway Performance Program (NHPP),0417000023S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN ALAMEDA AND SOLANO COUNTIES AT VARIOUS LOCATION INSTALL HIGHWAY PLANTING AND IRRIGATION SYSTEMS,2224000,|0|,|nan|,999,Statewide,nan,None,None,Install Planting and Irrigation Systems in Alameda,"Install Planting and Irrigation Systems in Alameda, part of the National Highway Performance Program (NHPP). (Federal Project ID: 000C533)."
194,44686,000C524,California,S MTC,YS32,Section 164 Penalties - Use for HSIP Activities,0414000003S,17|21|42,Construction Engineering|Safety|Training,ALAMEDA AND CONTRA COSTA COUNTIES AT VARIOUS LOCATIONS CROSSWALK SAFETY ENHANCEMENT,768800,|0|,|nan|,999,Statewide,nan,None,None,Safety Improvements in Alameda,"Safety Improvements in Alameda, part of the Section 164 Penalties - Use for HSIP Activities. (Federal Project ID: 000C524)."
195,44686,000C536,California,S MTC,Y001,National Highway Performance Program (NHPP),0416000036S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,SANTA CLARA AND ALAMEDA COUNTIES ON ROUTE 880 AND 80 AT VARIOUS LOCATIONS FROM ROUTE 880/237 CONNECTOR SEPARATION TO 98TH AVE OVER CROSSING AND ON RO,3070900,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Santa Clara,"Road Restoration & Rehabilitation in Santa Clara, part of the National Highway Performance Program (NHPP). (Federal Project ID: 000C536)."
238,44691,X019032,California,S FCOG,Y001,National Highway Performance Program (NHPP),0616000234S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,"FRESNO COUNTY IN FRESNO AND CLOVIS ON ROUTES 41, 99, 168 AND 180 AT VARIOUS LOCATIONS REPLACE AND UPGRADE TRAFFIC MANAGEMENT SYSTEM AND INSTALL FIBER",12798200,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Fresno County,"Road Restoration & Rehabilitation in Fresno County, part of the National Highway Performance Program (NHPP). (Federal Project ID: X019032)."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,44916,37Y6004,California,S BCAG,ER01,Emergency Supplement Funding,0320000006S,6,4R - Restoration & Rehabilitation,"BUTTE COUNTY ON ROUTE 70 AT VARIOUS LOCATIONS IMPORT BORROW, JACK WELDED AND CORRUGATED STEEL PIPE",1482990,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Butte County,"Road Restoration & Rehabilitation in Butte County, part of the Emergency Supplement Funding. (Federal Project ID: 37Y6004)."
959,44949,000C499,California,S SACOG,Y240,Surface Transportation Block Grant,0315000165S,17,Construction Engineering,"IN VARIOUS COUNTIES, ON VARIOUS ROUTES AT VARIOUS LOCATIONS. REPAIR AND INSTALL PERMANENT AUTOMATIC VEHICLE CLASSIFICATION (AVC) TRUCK DATA COLLECTIO",70800,|0|,|nan|,999,Statewide,nan,None,None,Construction Engineering Projects Statewide,"Construction Engineering Projects Statewide, part of the Surface Transportation Block Grant. (Federal Project ID: 000C499)."
1026,44966,X075048,California,S MTC,YS30,Highway Safety Improvement Program (HSIP),0415000221S,17|21,Constru

In [60]:
(agg>>filter(_.new_project_title.str.contains("Project ")))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
2,44581,31RA002,California,S AMBAG,ER01,Emergency Supplement Funding,0518000118S,16|43,Right of Way|Utilities,MONTEREY COUNTY NEAR BIG SUR 2.3 MILES NORTH OF CASTRO CANYON BRIDGE TO 0.8 MILE SOUTH OF BIG SUR RIVER BRIDGE. EMERGENCY PROJECT - PERMANENT RESTORA,2983400,|20|,|nan|,53,Monterey County,nan,None,None,Right of Way Project in Monterey County,"Right of Way Project in Monterey County, part of the Emergency Supplement Funding. (Federal Project ID: 31RA002)."
12,44586,31S3002,California,S NON-MPO,ER01,Emergency Supplement Funding,0117000214S,16,Right of Way,IN HUMBOLDT CO NEAR KORBEL FROM 1.3 MI EA OF TRUCK SCALE HOUSE TO 1.8 MI EA OF TRUCK SCALE HOUSE & FROM N FORK MAD RIVER BR TO 0.3 MI EA OF N FORK MA,25142,|2|,|nan|,23,Humboldt County,nan,None,None,Right of Way Project in Humboldt County,"Right of Way Project in Humboldt County, part of the Emergency Supplement Funding. (Federal Project ID: 31S3002)."
13,44586,31S4002,California,S NON-MPO,ER01,Emergency Supplement Funding,0117000211S,16,Right of Way,IN HUMBOLDT COUNTY NEAR WEITCHPEC ON SR 96 FROM PM 27.0 TO 27.4 WEST OF BLUFF CREEK WALL MAJOR DAMAGE RESTORATION - RIGHT OF WAY,33526,|2|,|nan|,23,Humboldt County,nan,None,None,Right of Way Project in Humboldt County,"Right of Way Project in Humboldt County, part of the Emergency Supplement Funding. (Federal Project ID: 31S4002)."
20,44588,31D6002,California,S NON-MPO,ER01,Emergency Supplement Funding,0218000164S,16,Right of Way,"NEAR JUNCTION CITY, AT DUTCH CREEK ROAD. OFF-SITE ENVIRONMENTAL MITIGATION AND PLANT ESTABLISHMENT FOR THE BIG FRENCH CREEK SLIDE EMERGENCY PROJECT E",627841,|2|,|nan|,105,Trinity County,nan,None,None,Right of Way Project in Trinity County,"Right of Way Project in Trinity County, part of the Emergency Supplement Funding. (Federal Project ID: 31D6002)."
53,44602,31TQ002,California,S MTC,ER01,Emergency Supplement Funding,0418000139S,16|43,Right of Way|Utilities,"IN SOLANO COUNTY, NEAR VACAVILLE, AT ALLENDALE ROAD;T CONSTRUCT ROCK SLOPE PROTECTION AT WASHOUT;",117209,|3|,|nan|,95,Solano County,nan,None,None,Right of Way Project in Solano County,"Right of Way Project in Solano County, part of the Emergency Supplement Funding. (Federal Project ID: 31TQ002)."
118,44627,5108202,Long Beach,L5108SCAG,Y301,Transportation Alternatives,0722000055L,44,Other,"DOWNTOWN LONG BEACH (BOUNDED BY MAGNOLIA AVE TO THE WEST, OCEAN BLVD TO THE SOUTH, 7TH STREET TO THE NORTH, AND ALAMITOS AVE TO THE EAST). NON-INFRAS",543000,|47|,|nan|,37,Los Angeles County,58,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Non-Infrastructure Project in Long Beach,"Non-Infrastructure Project in Long Beach, part of the Transportation Alternatives. (Federal Project ID: 5108202)."
129,44637,6084277,Metropolitan Transportation Commission,L6084MTC,Y230,Surface Transportation Block Grant,0422000334L,44,Other,REGIONWIDE - NINE COUNTY SAN FRANCISCO BAY AREA THE REGIONAL MAPPING DATA SERVICES SYSTEM WILL EXPOSE A SET OF PREDEFINED MAPPING-RELATED DATA APIS.(,1800000,|11|,|nan|,13,Multi-County,84,NON-RTPA,Metropolitan Transportation Commission,Mapping Project in the Metropolitan Transportation Commission Region,"Mapping Project in the Metropolitan Transportation Commission Region, part of the Surface Transportation Block Grant. (Federal Project ID: 6084277)."
130,44637,6084278,Metropolitan Transportation Commission,L6084MTC,Y230,Surface Transportation Block Grant,0422000335L,44,Other,"REGION WIDE - NINE COUNTY SAN FRANCISCO BAY AREA THE PROJECT WILL DEVELOP STANDARDS AND SYSTEM DESIGNS FOR FULLY HARMONIZED WAYF

In [61]:
(agg>>filter(_.county_name=="Statewide")>>filter(_.new_project_title.str.contains("in")))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
19,44587,39AZ001,California,S ER NONE,ER01,Emergency Supplement Funding,0119000058S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN DEL NORTE COUNTY AT VARIOUS LOCATIONS REPAIR ROADWAY AND CULVERT,2638100,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Del Norte County,"Road Restoration & Rehabilitation in Del Norte County, part of the Emergency Supplement Funding. (Federal Project ID: 39AZ001)."
163,44676,000C533,California,S MTC,Y001,National Highway Performance Program (NHPP),0417000023S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN ALAMEDA AND SOLANO COUNTIES AT VARIOUS LOCATION INSTALL HIGHWAY PLANTING AND IRRIGATION SYSTEMS,2224000,|0|,|nan|,999,Statewide,nan,None,None,Install Planting and Irrigation Systems in Alameda,"Install Planting and Irrigation Systems in Alameda, part of the National Highway Performance Program (NHPP). (Federal Project ID: 000C533)."
194,44686,000C524,California,S MTC,YS32,Section 164 Penalties - Use for HSIP Activities,0414000003S,17|21|42,Construction Engineering|Safety|Training,ALAMEDA AND CONTRA COSTA COUNTIES AT VARIOUS LOCATIONS CROSSWALK SAFETY ENHANCEMENT,768800,|0|,|nan|,999,Statewide,nan,None,None,Safety Improvements in Alameda,"Safety Improvements in Alameda, part of the Section 164 Penalties - Use for HSIP Activities. (Federal Project ID: 000C524)."
195,44686,000C536,California,S MTC,Y001,National Highway Performance Program (NHPP),0416000036S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,SANTA CLARA AND ALAMEDA COUNTIES ON ROUTE 880 AND 80 AT VARIOUS LOCATIONS FROM ROUTE 880/237 CONNECTOR SEPARATION TO 98TH AVE OVER CROSSING AND ON RO,3070900,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Santa Clara,"Road Restoration & Rehabilitation in Santa Clara, part of the National Highway Performance Program (NHPP). (Federal Project ID: 000C536)."
238,44691,X019032,California,S FCOG,Y001,National Highway Performance Program (NHPP),0616000234S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,"FRESNO COUNTY IN FRESNO AND CLOVIS ON ROUTES 41, 99, 168 AND 180 AT VARIOUS LOCATIONS REPLACE AND UPGRADE TRAFFIC MANAGEMENT SYSTEM AND INSTALL FIBER",12798200,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Fresno County,"Road Restoration & Rehabilitation in Fresno County, part of the National Highway Performance Program (NHPP). (Federal Project ID: X019032)."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,44910,000C518,California,S SJCOG,YS30,Highway Safety Improvement Program (HSIP),1016000021S,17,Construction Engineering,"MERCED, SAN JOAQUIN AND STANISLAUS COUNTIES AT VARIOUS LOCATIONS UPGRADE EXISTING GUARDRAIL AND HMA DIKES",22417,|0|,|nan|,999,Statewide,nan,None,None,Upgrade Guardrails in San Joaquin,"Upgrade Guardrails in San Joaquin, part of the Highway Safety Improvement Program (HSIP). (Federal Project ID: 000C518)."
894,44916,37Y6004,California,S BCAG,ER01,Emergency Supplement Funding,0320000006S,6,4R - Restoration & Rehabilitation,"BUTTE COUNTY ON ROUTE 70 AT VARIOUS LOCATIONS IMPORT BORROW, JACK WELDED AND CORRUGATED STEEL PIPE",1482990,|0|,|nan|,999,Statewide,nan,None,None,Road Restoration & Rehabilitation in Butte County,"Road Restoration & Rehabilitation in Butte County, part of the Emergency Supplement Funding. (Federal Project ID: 37Y6004)."
959,44949,000C499,California,S SACOG,Y240,Surface Transportation Block Grant,0315000165S,17,Construction Engineering,"IN VARIOUS COUNTIES, ON VARIOUS ROUTES A

## Program Code deep dive

In [62]:
(agg>>filter(_.program_code_description==("nan"))).program_code.value_counts()

Series([], Name: program_code, dtype: int64)

In [63]:
(agg>>filter(_.program_code_description==("nan")))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col


### Read in Old Codes


(currently used in script)

In [64]:
#check project codes list
codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))


In [65]:
len(codes)

101

In [66]:
codes.head()

,description,iija_program_code,new_description
0,Advanced Transportation Technologies and Innovative Mobility Deployment,Y44A,Advanced Transportation Technologies and Innovative Mobility Deployment
1,Applalachian Developmen Highway System,Y140,Applalachian Developmen Highway System
2,Border State Infrastructure,Y500,Border State Infrastructure
3,Bridge Formula Program,Y110,Bridge Formula Program
4,Bridge Formula Program Off-System Bridges,Y120,Bridge Formula Program


### New Codes

In [67]:
## read in new project codes:
new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of lst_IIJA_Code_20230510.xlsx"))

In [68]:
new_codes

,iija_code,description,program_name,iija_program_code
0,|BFP|,Bridge Formula Program,Bridge Formula Program,Y120
1,|CPF|,Community Project Funding Congressionally Directed Spending,Community Project Funding Congressionally Directed Spending,Y928
2,|RAISE|,National Infrastractur Investment (RAISE),National Infrastructure Investment (RAISE),RA02
3,|RAISE|,National Infrastractur Investment (RAISE),National Infrastructure Investment (RAISE),RA03
4,|TID|,Technology and Innovation Deployment - FMIS,Technology and Innovation Deployment,Y37H
...,...,...,...,...
108,|HRDP|,Highway Research and Deveopment Program,Highway Research and Development Program,Y448
109,|BRRP|,Bridge Replacement and Rehabilitation Program,Bridge Replacement and Rehabilitation Program,Y908
110,|TID|,Technology and Innovation Deployment - FMIS,Technology and Innovation Deployment,Y37G
111,|ER|,Emergency Suppliment Funding,Emergency Supplement Funding,ER01


#### Remove | from codes

In [69]:
# new_codes['iija_code_2'] = new_codes['iija_code'].str.split("|").str[0]
new_codes['iija_code_2'] = new_codes['iija_code'].str.replace("|","")

/tmp/ipykernel_1667/474405769.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [70]:
new_codes.sample(5)

,iija_code,description,program_name,iija_program_code,iija_code_2
39,|TRAIN|,On-the-Job Training,On the Job Skills Training,Y290,TRAIN
104,|TID|,Technology and Innovation Deployment,Technology and Innovation Deployment,Y376,TID
76,|BFP|,Bridge Formula Program,Bridge Formula Program,Y110,BFP
90,|TE|,Training and Education-FMIS,Training and Education,Y439,TE
1,|CPF|,Community Project Funding Congressionally Directed Spending,Community Project Funding Congressionally Directed Spending,Y928,CPF


#### Codes with ranges

In [71]:
new_codes>>filter(_.iija_program_code.str.contains("-"))

,iija_code,description,program_name,iija_program_code,iija_code_2
83,|IIJAPRH|,IIJA Puerto Rico Highway,IIJA Puerto Rico Highway,YP10 -YP30,IIJAPRH


In [72]:
# ## checking with the proj
# proj>>filter(_.program_code.str.contains("YP"))

In [73]:
# proj.program_code.value_counts()

#### Null codes

In [74]:
len(new_codes>>filter(_.iija_code.isna()))

0

In [75]:
null_program_codes = new_codes>>filter(_.iija_code.isna())

In [76]:
#download and send to contact! 
# null_program_codes.to_csv("lst_IIJA_Code_20230501_null.csv")

In [77]:
# null_program_codes

### Add acronym code for IIJA programs
https://stackoverflow.com/questions/4355201/creating-acronyms-in-python

In [78]:
# (proj>>filter(_.program_code==("ER01"), _.program_code_description.notnull())).sample()

In [79]:
# (proj>>filter(_.program_code==("ER01"), _.program_code_description.notnull())).sample()

In [80]:
df.head()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,county_name_title
0,44594,ER01,Emergency Supplement Funding,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,4,4R - No Added Capacity,68757.00,60870.23,L5904NON-MPO,5904.0,Humboldt County,nan,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
1,44594,ER01,Emergency Supplement Funding,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,15,Preliminary Engineering,207848.00,184008.19,L5904NON-MPO,5904.0,Humboldt County,nan,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
2,44594,ER01,Emergency Supplement Funding,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,17,Construction Engineering,139775.00,123742.50,L5904NON-MPO,5904.0,Humboldt County,nan,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
3,44594,ER01,Emergency Supplement Funding,4400046,01924731L,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,23,2,Active,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,4,4R - No Added Capacity,124980.90,110645.59,L5904NON-MPO,5904.0,Humboldt County,nan,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
4,44594,ER01,Emergency Supplement Funding,4400046,01924731L,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,23,2,Active,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,15,Preliminary Engineering,831.49,736.11,L5904NON-MPO,5904.0,Humboldt County,nan,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County


##### Check titles
moved to `_script_utils`

In [81]:
# sorted(list(agg.project_title_new.unique()))

In [82]:
# agg>>filter(_.implementing_agency.str.contains("California"))

In [83]:
# agg>>filter(_.project_title_new.str.contains("Caltrans"))

In [84]:
# len(agg)

In [85]:
# agg = utils.title_column_names(agg)

In [86]:
# agg.to_csv(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230315_ToDLA_agg.csv")

In [87]:
###test full. PASS title, no concat and 1241 rows
# full = utils.get_clean_data(df, full_or_agg = 'full')

In [88]:
# full = utils.title_column_names(full)

In [89]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")

In [90]:
# agg = to_snakecase(agg)

In [91]:
# test_df = agg>>filter(_.implementing_agency.str.contains('Unknown')
                      # | _.implementing_agency.str.contains("Caltrans")
                     # )

In [92]:
# test_df.sample()

In [93]:
# locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))

In [94]:
# locodes['agency_name'] = locodes['agency_name'].str.upper()

In [95]:
# locode_names = sorted(list(locodes['agency_name'].unique()), reverse=True)

In [96]:
# locode_names.remove('ROSS')

In [97]:
# locode_names.append("State Parks")

In [98]:
# locode_names

In [99]:
# pattern = '|'.join(locode_names)

In [100]:
# test_df.project_title.str.contains(pattern)

In [101]:
### https://stackoverflow.com/questions/68869434/create-an-pandas-column-if-a-string-from-a-list-matches-from-another-column
#test_df["agencies_found"] = test_df["project_title"].apply(lambda x: next((a for a in locode_names if a in x), ""))
# test_df["agencies_found2"] = test_df["project_title"].map(lambda s: next((name for name in locode_names if name in s), np.nan))

In [102]:
# test_df["agencies_found2"] = test_df["agencies_found2"].str.title()

In [103]:
# test_df

In [104]:
# agg>>filter(_["Project Title New"].str.contains('Unknown'))

In [105]:
# agg>>filter(_["implementing_agency"].str.contains('Unknown'))

## Next

#### Statewide 

In [106]:
len(agg>>filter(_.new_project_title.str.contains("Statewide")))

9

In [107]:
state = agg>>filter(_.new_project_title.str.contains("Statewide"))

In [108]:
state.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
311,44728,0010211,California,S NON-MPO,Y560,"Research, Development, and Technology Transfer (RD&T)",6500010147S,19,Research,"STATEWIDE STATE PLANNING AND RESEARCH PROGRAM, PART II FY 22/23",14493165,|0|,|nan|,999,Statewide,nan,None,None,Planning and Research Statewide,"Planning and Research Statewide, part of the Research, Development, and Technology Transfer (RD&T). (Federal Project ID: 0010211)."
711,44830,00PE023,California,S NON-MPO,YS30,Highway Safety Improvement Program (HSIP),00000PE023S,15,Preliminary Engineering,STATEWIDE STATEWIDE PRELIMINARY ENGINEERING 2022-23,41635714,|0|,|nan|,999,Statewide,nan,None,None,Preliminary Engineering Projects Statewide,"Preliminary Engineering Projects Statewide, part of the Highway Safety Improvement Program (HSIP). (Federal Project ID: 00PE023)."
656,44826,0010045,California,S NON-MPO,Y570,Safe and Accessible Transportation Options - SPR,7400010045S,18,Planning,"STATEWIDE STATE PLANNING AND RESEARCH PROGRAM, PART I FY 21/22",2275891,|0|,|nan|,999,Statewide,nan,None,None,Planning and Research Statewide,"Planning and Research Statewide, part of the Safe and Accessible Transportation Options - SPR. (Federal Project ID: 0010045)."
710,44830,00PE023,California,S NON-MPO,Y240,Surface Transportation Block Grant,00000PE023S,15,Preliminary Engineering,STATEWIDE STATEWIDE PRELIMINARY ENGINEERING 2022-23,58809712,|0|,|nan|,999,Statewide,nan,None,None,Preliminary Engineering Projects Statewide,"Preliminary Engineering Projects Statewide, part of the Surface Transportation Block Grant. (Federal Project ID: 00PE023)."
959,44949,000C499,California,S SACOG,Y240,Surface Transportation Block Grant,0315000165S,17,Construction Engineering,"IN VARIOUS COUNTIES, ON VARIOUS ROUTES AT VARIOUS LOCATIONS. REPAIR AND INSTALL PERMANENT AUTOMATIC VEHICLE CLASSIFICATION (AVC) TRUCK DATA COLLECTIO",70800,|0|,|nan|,999,Statewide,nan,None,None,Construction Engineering Projects Statewide,"Construction Engineering Projects Statewide, part of the Surface Transportation Block Grant. (Federal Project ID: 000C499)."


In [109]:
# #put into function

In [110]:
# locode_names = read_in_locodes()

In [111]:
# locode_names

In [112]:
#test

In [113]:
# state2 = find_alternative_name(state, 'project_title', "name_in_title")

In [114]:
# state2.sample(5)

In [115]:
# state2.sample()

In [116]:
# state2>>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.program_code_description, _.improvement_type_description,
#                                                       _.project_title, _.obligations_amount, _.county_name, _.county_name_title,
#                                                       _.project_title_new, _.new_description_col, _.name_in_title)

In [117]:
# state2>>filter(_.name_in_title==(""))>>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.program_code_description, _.improvement_type_description,
#                                                       _.project_title, _.obligations_amount, _.county_name, _.county_name_title,
#                                                       _.project_title_new, _.new_description_col, _.name_in_title)

In [118]:
# state3 = ((find_alternative_name(
#         (state2>>filter(_.name_in_title=="")), 'summary_recipient_defined_text_field_1_value', 'name_in_recp'))
#         >>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.program_code_description, _.improvement_type_description,
#                                                       _.project_title, _.obligations_amount, _.county_name, _.county_name_title,
#                                                       _.project_title_new, _.new_description_col, _.name_in_title, _.name_in_recp)
# )

In [119]:
# state3

#### County Codes

In [120]:
agg.county_code.value_counts()

37                     87
999                    76
19                     66
23                     59
65                     46
                       ..
55|97                   1
37|59                   1
75|85                   1
1|13|41|75|81|85|95     1
25                      1
Name: county_code, Length: 67, dtype: int64

In [121]:
agg>>filter(_.county_name.str.contains("Multi"))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
126,44635,6084275,Metropolitan Transportation Commission,L6084MTC,Y230,Surface Transportation Block Grant,0422000280L,44,Other,"FREMONT, RICHMOND, AND MARIN AND SONOMA COUNTIES, ALONG THE SMART CORRIDOR. BIKE SHARE CAPITAL PROGRAM (TC)",700000,|15|,|nan|,13,Multi-County,84,NON-RTPA,Metropolitan Transportation Commission,Bike Share Program in the Metropolitan Transportation Commission Region,"Bike Share Program in the Metropolitan Transportation Commission Region, part of the Surface Transportation Block Grant. (Federal Project ID: 6084275)."
129,44637,6084277,Metropolitan Transportation Commission,L6084MTC,Y230,Surface Transportation Block Grant,0422000334L,44,Other,REGIONWIDE - NINE COUNTY SAN FRANCISCO BAY AREA THE REGIONAL MAPPING DATA SERVICES SYSTEM WILL EXPOSE A SET OF PREDEFINED MAPPING-RELATED DATA APIS.(,1800000,|11|,|nan|,13,Multi-County,84,NON-RTPA,Metropolitan Transportation Commission,Mapping Project in the Metropolitan Transportation Commission Region,"Mapping Project in the Metropolitan Transportation Commission Region, part of the Surface Transportation Block Grant. (Federal Project ID: 6084277)."
130,44637,6084278,Metropolitan Transportation Commission,L6084MTC,Y230,Surface Transportation Block Grant,0422000335L,44,Other,"REGION WIDE - NINE COUNTY SAN FRANCISCO BAY AREA THE PROJECT WILL DEVELOP STANDARDS AND SYSTEM DESIGNS FOR FULLY HARMONIZED WAYFINDING, MAPPING, AND",991538,|11|,|nan|,13,Multi-County,84,NON-RTPA,Metropolitan Transportation Commission,Mapping Project in the Metropolitan Transportation Commission Region,"Mapping Project in the Metropolitan Transportation Commission Region, part of the Surface Transportation Block Grant. (Federal Project ID: 6084278)."
204,44686,6204134,Caltrans,S6204MTC,Y230,Surface Transportation Block Grant,0419000044L,15,Preliminary Engineering,"ALONG THE I-880 CORRIDOR SF BAY AREA: REGIONWIDE: IMPLEMENT A COLLECTIVE APPROACH TO FREEWAY OPERATIONS AND MANAGEMENT, INCLUDING COMMUNICATIONS NETW",1740000,|13|,|nan|,1,Multi-County,64,CT-ADMIN,CT-ADMIN,Caltrans Preliminary Engineering Projects,"Caltrans Preliminary Engineering Projects, part of the Surface Transportation Block Grant. (Federal Project ID: 6204134)."
268,44705,6084279,Metropolitan Transportation Commission,L6084MTC,Y230,Surface Transportation Block Grant,0422000413L,44,Other,SF BAY AREA: REGIONWIDE TECHNICAL ASSISTANCE MOBILITY HUB PILOT PROGRAM(TC),150000,|15|,|nan|,1,Multi-County,84,NON-RTPA,Metropolitan Transportation Commission,Pilot Program in the Metropolitan Transportation Commission Region,"Pilot Program in the Metropolitan Transportation Commission Region, part of the Surface Transportation Block Grant. (Federal Project ID: 6084279)."
325,44728,6205024,Caltrans,S6205SBCAG,YS30,Highway Safety Improvement Program (HSIP),0517000039L,21,Safety,ON SR 135 (BROADWAY) IN SANTA MARIA FROM FOSTER RD. TO PREISKER LN. INCREASE VISIBILITY OF TRAFFIC SIGNAL INDICATIONS BY INSTALLING ADDITIONAL TRAFFI,795900,|24|,|nan|,83,Multi-County,65,CT-ADMIN,CT-ADMIN,Install Traffic Signals in Santa Maria,"Install Traffic Signals in Santa Maria, part of the Highway Safety Improvement Program (HSIP). (Federal Project ID: 6205024)."
344,44736,6084282,Metropolitan Transportation Commission,L6084MTC,Y003,Projects to Reduce PM 2.5 Emissions,0422000469L,15,Preliminary Engineering,SF BAY AREA: VARIOUS BRIDGE CORRIDORS AND CORRIDOR APPROACHES CONDUCT PRELIMINARY ENGINEERING AND PLANNING STUDIES TO ADVANCE BAY BRIDGE FORWARD PROJ,12000000,|13|15|,|nan|,1,Multi-County,84,NON-RTPA,Metropolitan Transportation Commission,Preliminary Engineering Projects in t

In [122]:
agg.sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
1062,44978,5944100,Monterey County,L5944AMBAG,Y120,Bridge Formula Program,05930266L,14|17,Bridge Rehabilitation - No Added Capacity|Construction Engineering,BRADLEY ROAD BRIDGE AT SALINAS RIVER SCOUR PROTECTION (TC),4828000,|20|,|nan|,53,Monterey County,5944,Transportation Agency For Monterey County,Association of Monterey Bay Area Governments,Erosion Countermeasures in Monterey County,"Erosion Countermeasures in Monterey County, part of the Bridge Formula Program. (Federal Project ID: 5944100)."


### OTHER

In [123]:
agg.sample(10)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
557,44813,000C535,California,S MTC,YS32,Section 164 Penalties - Use for HSIP Activities,0416000123S,17|21,Construction Engineering|Safety,"ALAMEDA AND CONTRA COSTA COUNTIES AT VARIOUS LOCATIONS INSTALL CONCRETE BARRIERS, CRASH CUSHIONS AND GUARDRAILS.",1000000,|0|,|nan|,999,Statewide,nan,None,None,Install Guardrails in Alameda,"Install Guardrails in Alameda, part of the Section 164 Penalties - Use for HSIP Activities. (Federal Project ID: 000C535)."
1118,44993,5945103,Kings County,L5945KCAG,Y110,Bridge Formula Program,0616000178L,17|47,Construction Engineering|Bridge Preventive Maintenance,"PM00144 FOR 10 BRIDGES: 45C0006, 45C0015, 45C0033, 45C0044, 45C0081, 45C0095, 45C0096, 45C0101, 45C0113, AND 45C0114. BRIDGE PREVENTATIVE MAINTENANCE",1420906,|21|,|nan|,31,Kings County,5945,Kings County Association of Governments - RTPA,Kings County Association of Governments,Construction Engineering Projects in Kings County,"Construction Engineering Projects in Kings County, part of the Bridge Formula Program. (Federal Project ID: 5945103)."
153,44669,5940099,Mariposa County,L5940NON-MPO,Y233,Surface Transportation Block Grant,1012000240L,16,Right of Way,DOGTOWN ROAD OVER MAXWELL CREEK (BRIDGE 40C0038) BRIDGE REPLACEMENT (TC),50000,|4|,|nan|,43,Mariposa County,59,Mariposa County Transportation Commission,NON-MPO,Replace Bridge in Mariposa County,"Replace Bridge in Mariposa County, part of the Surface Transportation Block Grant. (Federal Project ID: 5940099)."
787,44879,6158111,Placer County Transportation Planning Agency,L6158SACOG,Y400,Congestion Mitigation & Air Quality Improvement,0323000052L,44,Other,FY 2023 CONGESTION MANAGEMENT PROGRAM IN PLACER COUNTY PROVIDE EDUCATIONAL AND OUTREACH EFFORTS REGARDING ALTERNATIVE TRANSPORTATION MODES TO EMPLOYE,50000,|1|4|,|nan|,61,Placer County,6158,Placer County Transportation Planning Agency,Sacramento Area Council Of Governments,Project in Placer County Transportation Planning Agency,"Project in Placer County Transportation Planning Agency, part of the Congestion Mitigation & Air Quality Improvement. (Federal Project ID: 6158111)."
192,44684,S245001,California,S TCAG,Y240,Surface Transportation Block Grant,0615000293S,6|11|17|42,4R - Restoration & Rehabilitation|Bridge Replacement - No Added Capacity|Construction Engineering|Training,IN TULARE COUNTY NEAR WOODLAKE AT YOKOHL CREEK AND AT KAWEAH RIVER BRIDGE REPLACE BRIDGES,11964097,|22|23|,|nan|,107,Tulare County,nan,None,None,Replace Bridge in Tulare County,"Replace Bridge in Tulare County, part of the Surface Transportation Block Grant. (Federal Project ID: S245001)."
459,44784,31PT001,California,S ER NONE,ER01,Emergency Supplement Funding,0317000219S,6|15|16|17,4R - Restoration & Rehabilitation|Preliminary Engineering|Right of Way|Construction Engineering,IN SIERRA COUNTY ON ROUTE 49 APPROXIMATELY 0.2 MILE SOUTH OF LAGER BEER SIDEHILL VIADUCT (#13 0023) INSTALL 3 SOLDIER PILE WALLS,1701353,|1|,|nan|,91,Sierra County,nan,None,None,Road Restoration & Rehabilitation in Sierra County,"Road Restoration & Rehabilitation in Sierra County, part of the Emergency Supplement Funding. (Federal Project ID: 31PT001)."
999,44953,P022028,California,S SCAG,YS30,Highway Safety Improvement Program (HSIP),1216000129S,17|21,Construction Engineering|Safety,ORANGE COUNTY IN SANTA ANA AND ORANGE FROM 0.2 MILE WEST OF BEDFORD ROAD OVERCROSSING TO 0.2 MILE WEST OF TUSTIN AVENUE UNDERCROSSING REPLACE CONC BA,3007500,|46|,|nan|,59,Orange County,nan,None,None,Safety Improvements in Orange County,"Safety Improvements in Orange County, part of the Highway Safety Improvement Program (HSI

#### Change CT District formatting

In [124]:
agg.district.value_counts()

|nan|    1120
Name: district, dtype: int64

In [125]:
agg.sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
118,44627,5108202,Long Beach,L5108SCAG,Y301,Transportation Alternatives,0722000055L,44,Other,"DOWNTOWN LONG BEACH (BOUNDED BY MAGNOLIA AVE TO THE WEST, OCEAN BLVD TO THE SOUTH, 7TH STREET TO THE NORTH, AND ALAMITOS AVE TO THE EAST). NON-INFRAS",543000,|47|,|nan|,37,Los Angeles County,58,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Non-Infrastructure Project in Long Beach,"Non-Infrastructure Project in Long Beach, part of the Transportation Alternatives. (Federal Project ID: 5108202)."


In [126]:
agg.congressional_district.value_counts()

|2|                                136
|1|                                 82
|0|                                 76
|4|                                 60
|3|                                 54
                                  ... 
|25|28|29|30|33|34|37|40|43|44|      1
|22|4|                               1
|2|51|                               1
|17|19|                              1
|45|48|                              1
Name: congressional_district, Length: 104, dtype: int64

In [127]:
# sb = agg.head()

In [128]:
# sb

In [129]:
# sb.district.value_counts()

In [130]:
# sb['district'] = sb['district'].str.replace(".0", "")

In [131]:
# sb

In [132]:
# def change_district_format(df, district_col):
#         district_map = ({'1.0':'01', '2.0':'02', '3.0':'03',
#                  '4.0': '04', '5.0': '05', '6.0':'06', '7.0':'07',
#                  '8.0':'08', '9.0':'09', '10.0':'10',
#                  '11.0':'11', '12.0':'12'})
        
#         df[district_col] = df[district_col].astype(str)
        
#         df[f"{district_col}2"] = df[district_col].map(district_map)
        
#         df[f"{district_col}3"] = '|' + df[f"{district_col}2"] + '|'
        
#         return df

In [133]:
# change_district_format(agg, "district")

#### Change ER Codes